In [0]:
# Unwanted files spark
spark.conf.set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol")
spark.conf.set("parquet.enable.summary-metadata", "false")
spark.conf.set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")

# Set account key
storage_account_name = "desafioabinbev"
account_key = "RApm9QtERW4cJlgK25W9sus968RiyP0Ua2CkLWjKn8nWA/H0pR1l+zTn8K03MtlcTftbrxp/5VyX+AStprx+jw=="
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", account_key)

In [0]:
def build_file_path(storage_account_name, container_name, blob_name):
    return f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/{blob_name}"

In [0]:
def read_json_to_df(file_path):
    try:
        return spark.read.json(file_path)
    except Exception as e:
        print(f"Error reading JSON file: {e}")
        return None

In [0]:
def write_df_to_container(df, output_path):
    try:
        df.write.mode("overwrite").partitionBy("country", "state").parquet(output_path)
        print("Data successfully written to the container in parquet format.")
    except Exception as e:
        print(f"Error writing dataframe to the container {e}")

In [0]:
from pyspark.sql.functions import col

def data_quality(df):
    try:
        if df.filter(col("id").isNull()).count() > 0:
            raise ValueError("Data quality check failed: null values found in the ID column")
        return True
    except ValueError as e:
        print(e)
        return False
    except Exception as e:
        print(f"Unexpected error during data quality check: {e}")
        return False

In [0]:
from datetime import datetime

storage_account_name = "desafioabinbev"
container_name_bronze = "abinbev-bronze"
container_name_silver = "abinbev-silver"
today = datetime.today().strftime('%Y%m%d')
blob_name = f"{today}/breweries_data.json"

file_path_bronze = build_file_path(storage_account_name, container_name_bronze, blob_name)

df = read_json_to_df(file_path_bronze)

if df and data_quality(df):
    display(df)
    print("Number of records:", df.count())

    output_path_silver = build_file_path(storage_account_name, container_name_silver, today)
    write_df_to_container(df, output_path_silver)
else:
    print("DataFrame is none or failed data quality. Writing operation canceled")

address_1,address_2,address_3,brewery_type,city,country,id,latitude,longitude,name,phone,postal_code,state,state_province,street,website_url
1716 Topeka St,null,null,micro,Norman,United States,5128df48-79fc-4f0f-8b52-d06be54d0cec,35.25738891,-97.46818222,(405) Brewing Co,4058160490,73069-8224,Oklahoma,Oklahoma,1716 Topeka St,http://www.405brewing.com
407 Radam Ln Ste F200,null,null,micro,Austin,United States,9c5a66c8-cc13-416f-a5d9-0a769c87d318,null,null,(512) Brewing Co,5129211545,78745-1197,Texas,Texas,407 Radam Ln Ste F200,http://www.512brewing.com
8100 Washington Ave,null,null,micro,Mount Pleasant,United States,34e8c68b-6146-453f-a4b9-1f6cd99a5ada,42.72010826899558,-87.88336350209435,1 of Us Brewing Company,2624847553,53406-3920,Wisconsin,Wisconsin,8100 Washington Ave,https://www.1ofusbrewing.com
1501 E St,null,null,large,San Diego,United States,ef970757-fe42-416f-931d-722451f1f59c,32.714813,-117.129593,10 Barrel Brewing Co,6195782311,92101-6618,California,California,1501 E St,http://10barrel.com
62970 18th St,null,null,large,Bend,United States,6d14b220-8926-4521-8d19-b98a2d6ec3db,44.08683531,-121.281706,10 Barrel Brewing Co,5415851007,97701-9847,Oregon,Oregon,62970 18th St,http://www.10barrel.com
1135 NW Galveston Ave Ste B,null,null,large,Bend,United States,e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2,44.0575649,-121.3288021,10 Barrel Brewing Co,5415851007,97703-2465,Oregon,Oregon,1135 NW Galveston Ave Ste B,null
1411 NW Flanders St,null,null,large,Portland,United States,e432899b-7f58-455f-9c7b-9a6e2130a1e0,45.5259786,-122.6855056,10 Barrel Brewing Co,5032241700,97209-2620,Oregon,Oregon,1411 NW Flanders St,http://www.10barrel.com
62950 NE 18th St,null,null,large,Bend,United States,9f1852da-c312-42da-9a31-097bac81c4c0,44.0912109,-121.2809536,10 Barrel Brewing Co - Bend Pub,5415851007,97701,Oregon,Oregon,62950 NE 18th St,null
826 W Bannock St,null,null,large,Boise,United States,ea4f30c0-bce6-416b-8904-fab4055a7362,43.618516,-116.202929,10 Barrel Brewing Co - Boise,2083445870,83702-5857,Idaho,Idaho,826 W Bannock St,http://www.10barrel.com
2620 Walnut St,null,null,large,Denver,United States,1988eb86-f0a2-4674-ba04-02454efa0d31,39.7592508,-104.9853655,10 Barrel Brewing Co - Denver,7205738992,80205-2231,Colorado,Colorado,2620 Walnut St,null
